<a href="https://colab.research.google.com/github/Manishkumarsingh41/Manish_singh_DataScience/blob/main/rag_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Install dependencies

In [4]:
!pip install faiss-cpu sentence-transformers transformers PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 17.6 MB/s eta 0:00:00


In [5]:
!pip install pycryptodome


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 13.8 MB/s eta 0:00:00


# Step 2: Load PDFs

In [6]:
import os
from PyPDF2 import PdfReader

folder_path = "/content/Task2doc"

def load_pdfs(folder_path):
    docs = []
    for file in os.listdir(folder_path):
        if file.endswith(".pdf"):
            try:
                reader = PdfReader(os.path.join(folder_path, file))
                if reader.is_encrypted:
                    print(f"Skipping encrypted file: {file}")
                    continue
                text = ""
                for page in reader.pages:
                    text += page.extract_text() + "\n"
                docs.append({'title': file, 'text': text})
            except Exception as e:
                print(f"Error reading {file}: {e}")
    return docs

docs = load_pdfs(folder_path)
print(f"Loaded {len(docs)} documents")


Skipping encrypted file: 172-65231ma.pdf
Loaded 10 documents


# Step 3: Chunk the documents

In [7]:
def chunk_text(text, chunk_size=500, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

all_chunks = []
chunk_sources = []

for doc in docs:
    chunks = chunk_text(doc['text'])
    all_chunks.extend(chunks)
    chunk_sources.extend([doc['title']] * len(chunks))

print(f"Created {len(all_chunks)} chunks from {len(docs)} documents")


Created 91 chunks from 10 documents


# Step 4: Create embeddings and index with FAISS

In [8]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load embedding model
embed_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Compute embeddings
embeddings = embed_model.encode(all_chunks, convert_to_numpy=True)

# Build FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

print("FAISS index ready with embeddings")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

FAISS index ready with embeddings


#Step 5: Define query function

In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# Load a small, free LLM
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
llm_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

qa_pipeline = pipeline("text2text-generation", model=llm_model, tokenizer=tokenizer)


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu


In [10]:
def query_rag(question, top_k=3, max_chunk_words=100):
    q_emb = embed_model.encode([question])
    D, I = index.search(q_emb, top_k)

    retrieved_chunks = [all_chunks[i] for i in I[0]]
    sources = [chunk_sources[i] for i in I[0]]

    if not retrieved_chunks:
        return "No relevant information found."

    truncated_chunks = [" ".join(chunk.split()[:max_chunk_words]) for chunk in retrieved_chunks]
    context = "\n\n".join([f"[Source: {src}] {chunk}" for src, chunk in zip(sources, truncated_chunks)])

    prompt = f"""
You are an expert technical assistant.
Answer the question clearly and concisely, using ONLY the context below.
Interpret the information to directly answer the question.
Provide a short explanation in simple language.

Context:
{context}

Question: {question}
Answer:
"""
    # Generate answer
    raw_answer = qa_pipeline(prompt, max_new_tokens=200)[0]['generated_text']

    # Clean output
    answer = raw_answer.strip()

    # Append top-k source (first retrieved chunk)
    if sources:
        answer += f" [Source: {sources[0]}]"

    return answer


In [11]:
sensitive_keywords = [
    "hack", "exploit", "illegal", "attack", "bypass",
    "personal info", "password", "credit card"
]

def is_sensitive(query):
    query_lower = query.lower()
    for word in sensitive_keywords:
        if word in query_lower:
            return True
    return False

# Usage in query_rag
question = "How to hack the system?"
if is_sensitive(question):
    print("Query blocked: Sensitive or unsafe content detected.")
else:
    answer = query_rag(question)
    print(answer)


Query blocked: Sensitive or unsafe content detected.


In [12]:
test_data = [
    {"question": "What does a sudden draft drop indicate?", "answer": "Cyclone overflowing"},
    {"question": "What is the optimal spigot size?", "answer": "Small enough to allow underflow solids, big enough for air core"}
]


In [13]:
def evaluate_rag(test_data, top_k=3):
    results = []
    for item in test_data:
        q_emb = embed_model.encode([item["question"]])
        D, I = index.search(q_emb, top_k)
        retrieved_sources = [chunk_sources[i] for i in I[0]]

        # Simple heuristic: check if gold answer appears in any retrieved chunk
        relevant_found = any(item["answer"].lower() in all_chunks[i].lower() for i in I[0])
        results.append({
            "question": item["question"],
            "retrieved_sources": retrieved_sources,
            "relevant_found": relevant_found
        })
    return results

eval_results = evaluate_rag(test_data)
print(eval_results)


[{'question': 'What does a sudden draft drop indicate?', 'retrieved_sources': ['SOP Cyclone-22022023.pdf', 'sect7a.pdf', 'Parker_DustHog_C_Series_Cyclone_Dust_Collector_User_Guide.pdf'], 'relevant_found': False}, {'question': 'What is the optimal spigot size?', 'retrieved_sources': ['sect7a.pdf', 'Parker_DustHog_C_Series_Cyclone_Dust_Collector_User_Guide.pdf', 'D13.pdf'], 'relevant_found': False}]


# Step 6: Test the RAG system

In [16]:
question = "What does a sudden draft drop indicate?"
answer = query_rag(question)
print("Answer:", answer)


Answer: A sudden draft drop indicates that the cyclone is overflowing. [Source: SOP Cyclone-22022023.pdf]
